# Part 4: Tool Calling - Giving Your LLM Research Superpowers

<div class="alert alert-success">
<h3>🎯 Workshop Goals (Part 4)</h3>
<p>In this final hour, we will give our LLM the ability to interact with the outside world. This is the key to unlocking its potential as a true research assistant. You will learn:</p>
<ul>
<li><strong>Why a Researcher Needs Tools:</strong> Understand the inherent limitations of LLMs, such as knowledge cutoffs and the inability to access your private data or the live internet.</li>
<li><strong>The Tool Calling Loop:</strong> Master the fundamental request-execute-respond pattern that powers all modern AI agents.</li>
<li><strong>Building a Research Assistant:</strong> Create a set of tools that could help automate a social science research workflow.</li>
</ul>
</div>

1. The Researcher's Dilemma: The Brain in the Jar
So far, we've treated the LLM as an incredibly knowledgeable, fast, and reliable data processor. You give it text, and it gives you back perfectly structured data. This is a powerful skill for tasks like the COVID-19 narrative analysis we've discussed.

But for a researcher, this is only half the battle. Your work isn't static; it's a dynamic process. This exposes two major limitations of a standard LLM:

1. **The Knowledge Cutoff**: The model's knowledge stops the moment its training was completed. It cannot read the latest paper published on arXiv yesterday, access the most recent census data, or know about a news event that happened this morning. Its knowledge is a fixed library, not a live internet connection.
2. **The Inability to Take Action**: The LLM is isolated. It can't access your local files, query your university's research database, or even run a simple statistical calculation. It can write a script to analyze your data, but it can't run it.

Imagine you have a brilliant research assistant who is locked in the university's library archives from two years ago. They can read and synthesize any book in that room with superhuman speed, but they can't use a web browser, make a phone call, or open an Excel spreadsheet on your computer.

Tool calling is the act of giving that brilliant assistant a laptop and an internet connection. It connects the "brain" of the LLM to the "hands" of real-world tools, allowing it to overcome these limitations and become a true partner in your research workflow.

### How it Works: The Tool Calling Loop
At its core, tool calling is a multi-step conversation between your code and the LLM. It's not a single request and response, but a sequence of exchanges. We call this the Tool Calling Loop.

Let's make this concrete with a simple, non-research example: asking for the current time. The LLM doesn't have a live clock, so it will need a tool for that.

#### Step 1: Define Your Tool in Python
First, we write a standard Python function that does the thing we need. This function is our "tool." It's just regular code that runs on your machine, not in the LLM.

In [9]:
# We'll need our client and some other libraries
import json
from datetime import datetime
from openai import OpenAI
from pprint import pprint

# Read the API_KEY
with open('API_KEY.txt', 'r') as file:
    API_KEY = file.read() 

client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=API_KEY,
)

In [4]:
# Define our tool. A simple Python function to get the current time.
def get_current_time():
    """Gets the current local time."""
    # In a real application, this could be doing anything:
    # querying a database, calling another API, reading a file, etc.
    print("---  TOOL EXECUTED: get_current_time() ---")
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

get_current_time()  # Test the tool works

---  TOOL EXECUTED: get_current_time() ---


'2025-10-08 23:45:14'

#### Step 2: Describe the Tool to the LLM

Next, we need to create a "manual" or "menu" of our available tools for the LLM. We define this using a **specific JSON structure**. The description is the most important part! It's how the LLM decides when to use your tool. It should be clear and descriptive.

In [5]:
# This is the "menu" of tools we will offer the LLM.
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_time",
            "description": "Use this function to get the current date and time.",
            "parameters": { # This tool takes no parameters
                "type": "object",
                "properties": {},
            },
        },
    }
]

#### Step 3: The First API Call - The LLM Asks for a Tool

Now, we make our first API call. This is where the magic happens. The structure of the call is very similar to what you've seen before, but with two new, crucial parameters: `tools` and `tool_choice`.

- `tools`: This is where we pass the "menu" of available tools that we defined in the previous step. It's a list of JSON objects describing each function the model is allowed to request.
- `tool_choice`: This parameter controls how the model uses the tools.
    - "auto" (the default): The model decides for itself whether to use a tool or not based on the user's prompt. This is what you'll use most of the time.
    - "none": This forces the model to not use any tools and just respond with a standard message.
    - {"type": "function", "function": {"name": "my_function"}}: This forces the model to use a specific function.

This `tools` parameter is a core feature of the OpenAI API, and its standardization has been adopted by many other providers, including OpenRouter, making it a key skill to learn. You can read the complete, official documentation on this feature for more details.

Official Documentation: [OpenAI API Reference - Tool Calling](https://platform.openai.com/docs/guides/function-calling)

Let's make the call. Notice the response we get back. The LLM doesn't answer the question directly. Instead, it asks us to run a tool.

In [14]:
# The user's question that requires a tool
messages = [{"role": "user", "content": "What time is it?"}]

# Make the first API call, providing the tools menu
response = client.chat.completions.create(
    model="meta-llama/llama-3.3-8b-instruct:free",
    messages=messages,
    tools=tools,
)

In [15]:
# Inspect the response
response_message = response.choices[0].message
pprint(response_message.model_dump())  # Pretty-print the full response

{'annotations': None,
 'audio': None,
 'content': '',
 'function_call': None,
 'reasoning': None,
 'refusal': None,
 'role': 'assistant',
 'tool_calls': [{'function': {'arguments': '{}', 'name': 'get_current_time'},
                 'id': '94800ed9-2b25-4f6e-9e77-2fb69b67fe48',
                 'index': 0,
                 'type': 'function'}]}


🔔 Question: Look at the response_message output. What is the content of the message? What is the value of the tool_calls field?

You'll see that the message content is null. The LLM hasn't said anything. Instead, it has returned a `tool_calls` object. This is an instruction from the LLM to your code, saying: "I need to answer the user, but first, please run the `get_current_time` function for me."

#### Step 4: Your Code Executes the Tool and Sends Back the Result

In [ ]:
# First, append the LLM's instruction to call a tool to our message history
messages = [{"role": "user", "content": "What time is it?"}] # Our original user question
messages.append(response_message)

# Get the ID of the tool call
tool_call_id = response_message.tool_calls[0].id

# Call our actual Python function
tool_output = get_current_time()

# Now, append the *output* of our function to the message history
# This tells the model what the result of its requested tool call was.
messages.append(
    {
        "tool_call_id": tool_call_id,
        "role": "tool",
        "name": "get_current_time",
        "content": tool_output, # The actual result from our Python function
    }
)

print("--- Current Message History ---")
for msg in messages:
    pprint(msg)
    print()

---  TOOL EXECUTED: get_current_time() ---
--- Current Message History ---
{'content': 'What time is it?', 'role': 'user'}

ChatCompletionMessage(content='', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='94800ed9-2b25-4f6e-9e77-2fb69b67fe48', function=Function(arguments='{}', name='get_current_time'), type='function', index=0)], reasoning=None)

{'content': '2025-10-08 23:56:54',
 'name': 'get_current_time',
 'role': 'tool',
 'tool_call_id': '94800ed9-2b25-4f6e-9e77-2fb69b67fe48'}



#### Step 5: The Second API Call - The LLM Gives the Final Answer
Now that the LLM has the real-time information it needed, we make one final API call. This time, the model uses the context from the tool's output to generate a natural, human-readable answer to the user's original question.



In [26]:
# Make the second API call with the complete message history
second_response = client.chat.completions.create(
    model="alibaba/tongyi-deepresearch-30b-a3b:free",
    messages=messages,
)

In [27]:

# Print the final answer
final_answer = second_response.choices[0].message.content
print("\\n--- Final Answer from LLM ---")
print(final_answer)

\n--- Final Answer from LLM ---


The current time is **23:56:54** on **2025-10-08**.


⚠️ **Warning:** Notice we've switched models here from `mistral-instruct` in the previous parts of the workshop to `tongyi-deepresearch-30b`. This is becuase some models don't support tool calling. On OpenRouter's website you can filter by models that support tool calling. Always ensure the model you're using can, otherwise this will not work. 


#### 🥊 [Challenge] Your First Research Tool: The Paper Fetcher (10 mins)
Now it's your turn to build the entire tool-calling loop from scratch.

**Your Goal**: Create a tool that allows the LLM to look up information about a (fake) academic paper based on its ID. This simulates a common research task: querying a database like arXiv, PubMed, or JSTOR.

**Your Task**: You will need to write the code for all 5 steps of the loop to answer the user's question: "Can you tell me the title and author of the paper 2305.15334?"

In [ ]:
import json

# --- Step 1: Define Your Tool in Python ---
# This function simulates fetching data from an academic database.
# It should take one argument: 'paper_id' (a string).
def get_paper_details(paper_id: str):
    """
    Gets the title and author for a given paper ID from a mock database.
    """
    print(f"--- TOOL EXECUTED: Searching for paper {paper_id} ---")
    
    # A mock database of academic papers
    mock_database = {
        "2305.15334": {
            "title": "The Role of Social Media in Political Polarization",
            "author": "Dr. Eleanor Vance",
            "year": 2023,
        },
        "1706.03762": {
            "title": "Attention Is All You Need",
            "author": "Ashish Vaswani et al.",
            "year": 2017,
        }
    }
    
    paper_info = mock_database.get(paper_id, "Paper not found.")
    return json.dumps(paper_info)


# --- Step 2: Describe the Tool to the LLM ---
# Create the 'tools' list with the definition for 'get_paper_details'.
# Make sure you correctly define the 'parameters' the function expects!
# It should have one required parameter: 'paper_id' of type 'string'.


tool = [
    {
        "type": "function",
        "function": {
            "name": "get_paper_details",
            "description": "Fetch details about a specific academic paper.",
            "parameters": {
                "type": "object",
                "properties": {
                    "paper_id": {
                        "type": "string",
                        "description": "The ID of the paper to retrieve."
                    }
                },
                "required": ["paper_id"]
            }
        }
    }
]


# --- Step 3, 4, and 5: Execute the Full Loop ---
# Write the code to handle the entire conversation.

# 1. Define the initial message list with the user's prompt.
user_prompt = "Can you tell me the title and author of the paper 2305.15334?"
messages = [{"role": "user", "content": user_prompt}]

# 2. Make the first API call to get the tool_calls object.
response = client.chat.completions.create(
    model="meta-llama/llama-3.3-8b-instruct:free",
    messages=messages,
    tools=tool,
)
response_message = response.choices[0].message

# 3. Check for tool_calls, execute the function, and append the results to the messages list.
# (This part will be more complex than the time example, as you need to get the arguments!)

if response_message.tool_calls:
    # Extract the arguments for the tool call
    tool_args = response_message.tool_calls[0].arguments
    paper_id = tool_args.get("paper_id")

    # Call the function with the extracted arguments
    tool_response = get_paper_details(paper_id)

    # Append the tool response to the messages list
    messages.append({"role": "user", "content": tool_response})

# 4. Make the second API call to get the final, natural language answer.
second_response = client.chat.completions.create(
    model="meta-llama/llama-3.3-8b-instruct:free",
    messages=messages
)
print(second_response.choices[0].message.content)

#### What's Next? Handling Multiple Tools

So far, our agent has used one tool to answer one question. But a real research workflow is more complex. You might need to ask a question that requires multiple pieces of information from different sources.

For example: "Can you find the title of paper 2305.15334 and also find its citations on Google Scholar?"

The LLM is smart enough to recognize this. Instead of making you go back and forth for each piece of information, it can request all the tools it needs in a single turn. This is called parallel tool calling.

When this happens, the response_message.tool_calls object will no longer be a single item, but a list of tool calls. Your code needs to be ready to loop through this list, execute each requested tool, and gather all the results before sending them back to the LLM.

#### Example: A Multi-Tool Research Assistant
Let's add a second tool to our paper fetcher and see how to handle a parallel request.

In [31]:
# We'll use our get_paper_details function from before.

# Here is our new tool. It simulates another research task.
def get_citations_for_paper(paper_id: str):
    """Gets the number of citations for a paper from a mock database."""
    print(f"--- TOOL EXECUTED: Getting citations for {paper_id} ---")
    mock_citation_database = {
        "2305.15334": 121,
        "1706.03762": 85000,
    }
    citations = mock_citation_database.get(paper_id, 0)
    return json.dumps({"paper_id": paper_id, "citation_count": citations})

# Now, our 'tools' menu has two functions available.
tools = [
    {
        "type": "function", "function": {
            "name": "get_paper_details",
            "description": "Looks up the title and author of an academic paper given its unique ID.",
            "parameters": {"type": "object", "properties": {"paper_id": {"type": "string"}}, "required": ["paper_id"]}
        }
    },
    {
        "type": "function", "function": {
            "name": "get_citations_for_paper",
            "description": "Finds the number of citations for a given paper ID.",
            "parameters": {"type": "object", "properties": {"paper_id": {"type": "string"}}, "required": ["paper_id"]}
        }
    }
]

Now, watch how we adapt the loop to handle multiple tool calls. The key change is that we now **loop through** response_message.tool_calls.

In [ ]:
# A more complex prompt that requires BOTH tools.
user_prompt = "Please get me the title of paper 2305.15334 and also find out how many citations it has."
messages = [{"role": "user", "content": user_prompt}]

# 1. First API call
response = client.chat.completions.create(
    model="meta-llama/llama-3.3-8b-instruct:free", 
    messages=messages, 
    tools=tools,
    tool_choice="auto"
)
response_message = response.choices[0].message
messages.append(response_message)

In [33]:
# Sanity check: print the response to see the tool calls
pprint(response_message.model_dump())

{'annotations': None,
 'audio': None,
 'content': '',
 'function_call': None,
 'reasoning': None,
 'refusal': None,
 'role': 'assistant',
 'tool_calls': [{'function': {'arguments': '{"paper_id":"2305.15334"}',
                              'name': 'get_paper_details'},
                 'id': 'b644eae5-8500-4f09-aa39-8351056c1e4b',
                 'index': 0,
                 'type': 'function'},
                {'function': {'arguments': '{"paper_id":"2305.15334"}',
                              'name': 'get_citations_for_paper'},
                 'id': 'f5623cfb-2e95-42b0-8b2e-f204f20ae898',
                 'index': 1,
                 'type': 'function'}]}


We can confirm that it has called both tools as expected!

In [34]:
# 2. Execute ALL requested tools
if response_message.tool_calls:
    print(f"--- LLM requested {len(response_message.tool_calls)} tools ---")
    
    # This is the key change: We loop through each tool call requested by the model
    for tool_call in response_message.tool_calls:
        function_name = tool_call.function.name
        function_args = json.loads(tool_call.function.arguments)
        
        if function_name == "get_paper_details":
            tool_output = get_paper_details(paper_id=function_args.get("paper_id"))
        elif function_name == "get_citations_for_paper":
            tool_output = get_citations_for_paper(paper_id=function_args.get("paper_id"))
        
        # We append one message to the history FOR EACH tool call.
        messages.append({
            "tool_call_id": tool_call.id,
            "role": "tool",
            "name": function_name,
            "content": tool_output,
        })

--- LLM requested 2 tools ---
--- TOOL EXECUTED: Searching for paper 2305.15334 ---
--- TOOL EXECUTED: Getting citations for 2305.15334 ---


In [37]:
# Let's verify our message history so far after adding the messages from both tool calls in the cell above
print("--- Complete Message History ---")
pprint(messages)

--- Complete Message History ---
[{'content': 'Please get me the title of paper 2305.15334 and also find out '
             'how many citations it has.',
  'role': 'user'},
 ChatCompletionMessage(content='', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='b644eae5-8500-4f09-aa39-8351056c1e4b', function=Function(arguments='{"paper_id":"2305.15334"}', name='get_paper_details'), type='function', index=0), ChatCompletionMessageFunctionToolCall(id='f5623cfb-2e95-42b0-8b2e-f204f20ae898', function=Function(arguments='{"paper_id":"2305.15334"}', name='get_citations_for_paper'), type='function', index=1)], reasoning=None),
 {'content': '{"title": "The Role of Social Media in Political Polarization", '
             '"author": "Dr. Eleanor Vance", "year": 2023}',
  'name': 'get_paper_details',
  'role': 'tool',
  'tool_call_id': 'b644eae5-8500-4f09-aa39-8351056c1e4b'},
 {'content': '{"paper_id": "2305.15334", 

In [38]:
# 3. Second API call for the final answer
second_response = client.chat.completions.create(model="alibaba/tongyi-deepresearch-30b-a3b:free", messages=messages)

In [39]:
print(f"\\n--- Final LLM Answer ---\\n{second_response.choices[0].message.content}")

\n--- Final LLM Answer ---\n

The paper with the ID **2305.15334** is titled:  
**"The Role of Social Media in Political Polarization"** by Dr. Eleanor Vance (2023).  

It has received **121 citations** so far.


### 🥊 Grand Finale Challenge: The Narrative Standardization Agent (20 mins)


This is our final task. It will bring together everything you have learned to build a practical research assistant for the exact problem we started with: thematic coding of COVID-19 narratives.

##### The Research Context
Let's revisit our goal. We are social scientists with thousands of raw, unstructured narratives.

A typical raw narrative (Our Input):
> "I was working at the hospital in the South Bronx, and we were running out of everything. It felt hopeless. Every day was a mix of profound sadness for the patients we lost and this incredible solidarity with my fellow nurses. We were all in it together."

For this data to be useful, we need to process it into a clean, standardized format. This requires a clear "codebook" for our analysis and a defined structure for the output.

For this challenge we want to identify:
- The emotions expressed in each story
- Mentions of material conditions (e.g. PPE shortages)
- Instances of collective solidarity or isolation
- Themes of grief, duty, or burnout
- (New) Extract the location in the story and convert it to a location ISO Code For Categorization (South Bronx -> Bronx -> NYC-BX)

##### Your Task Part 1: Build the Pydantic "Codebook"

Before we can build the agent, we must define the structure of our final output. This is your first task. You will create the Pydantic models that will serve as the blueprint for our data.

    A) Create the Emotion Codebook

In qualitative research, a codebook ensures consistency. An `Enum` is the perfect way to enforce a strict codebook.

Your task: Create an `Enum` class called EmotionCode that allows for only the following string values: "Sadness", "Solidarity", "Hopelessness", "Fear", and "Grief".

In [ ]:
from pydantic import BaseModel, Field
from enum import Enum
from typing import List
import json

# --- Your Code Below ---
# 1. Define the EmotionCode Enum here.
# class EmotionCode(str, Enum):
#     SADNESS = "Sadness"
#     ...

class EmotionCode(str, Enum):
    SADNESS = "Sadness"
    SOLIDARITY = "Solidarity"
    HOPELESSNESS = "Hopelessness"
    FEAR = "Fear"
    GRIEF = "Grief"

    B) Create the Final Schema

Now, using your EmotionCode enum and the skills you learned in Part 3, define the final data structure.

Your task: Create a Pydantic BaseModel called CodedNarrative. It should have the following fields:
- summary: A required str.
- emotions: A required List of EmotionCode enums.
- material_conditions: A required List of str.
- primary_borough: A required str.

In [ ]:
# --- Your Code Below ---
# 2. Define the CodedNarrative BaseModel here.
# It should use the EmotionCode Enum you created above.
class CodedNarrative(BaseModel):
    summary: str = Field(..., description="A brief summary of the narrative.")
    emotions: List[EmotionCode] = Field(..., description="A list of emotions expressed in the narrative.")
    material_conditions: List[str] = Field(..., description="Description of the material conditions.")
    primary_borough: str = Field(..., description="The primary borough associated with the narrative.")

##### Your Task Part 2: Build the Agent
Now that you have your output schema, you can build the agent that will produce it. I've provided the custom tool for you. Your job is to create the tool's schema and write the full loop.

In [ ]:
# --- I've provided this tool for you ---
def get_borough_iso_code(borough_name: str):
    """
    Takes a standardized NYC borough name and returns its official ISO-like code.
    """
    print(f"--- TOOL: Getting ISO code for: '{borough_name}' ---")
    # This mock database maps a standardized borough name to a code.
    borough_code_database = {
        "Bronx": "NYC-BX",
        "Queens": "NYC-QN",
        "Manhattan": "NYC-MN",
        "Brooklyn": "NYC-BK",
        "Staten Island": "NYC-SI"
    }
    code = borough_code_database.get(borough_name, "Invalid Borough")
    return json.dumps({"iso_code": code})


# 3. DEFINE THE TOOL SCHEMA
# Create the 'tools' list with the JSON definition for the get_borough_iso_code function.
# Its single parameter should be 'borough_name'.
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_borough_iso_code",
            "description": "Gets the official ISO-like code for a given NYC borough name.",
            "parameters": {
                "type": "object",
                "properties": {
                    "borough_name": {
                        "type": "string",
                        "description": "The standardized name of the NYC borough."
                    }
                },
                "required": ["borough_name"]
            }
        }
    }
]

In [ ]:
# 4. BUILD THE AGENT
# The raw narrative to be processed
raw_narrative = (
    "I was working at the hospital in the South Bronx, and we were running out of everything. It felt hopeless. "
    "Every day was a mix of profound sadness for the patients we lost and this incredible solidarity with my fellow nurses."
)

# The System Prompt is the "brain" of our agent. Note the two-step instruction for location.
system_prompt = (
    "You are a social science research assistant. Your task is to thematically code a raw narrative about experiences in NYC during COVID-19. "
    "First, carefully read the narrative to identify the NYC borough. You must standardize fuzzy locations (e.g., 'South Bronx') into a proper borough name (e.g., 'Bronx'). "
    "Then, use the `get_borough_iso_code` tool with that standardized borough name. "
    "Next, analyze the text to determine a one-sentence summary, the emotions expressed (which must conform to the provided EmotionCode enum), "
    "and any material conditions mentioned. "
    "Finally, combine all this information into a single JSON object that perfectly matches the `CodedNarrative` schema."
)

# Execute the FULL tool-calling loop.
# It should end with a final call using .parse() and your CodedNarrative model to guarantee the output structure.

# --- Your Code Below ---
# 1. Start the conversation history
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": raw_narrative}
]

# 2. First API Call: The model will reason and ask to use the tool
print("--- 1. Making first API call to request tool ---")
response = client.chat.completions.create(
    model="meta-llama/llama-3.3-8b-instruct:free",
    messages=messages,
    tools=tools,
)

response_message = response.choices[0].message
messages.append(response_message)

# 3. Execute the tool the model requested
if response_message.tool_calls:
    tool_call = response_message.tool_calls[0]
    function_name = tool_call.function.name
    
    # The model correctly infers that "South Bronx" should be "Bronx"
    function_args = json.loads(tool_call.function.arguments)
    borough_arg = function_args.get("borough_name")
    
    # Call our Python function with the argument from the model
    function_response = get_borough_iso_code(borough_name=borough_arg)
    
    # Append the tool's output to the conversation. Hint: you need (tool_call_id, role, name, content)
    messages.append({
        "tool_call_id": tool_call.id,
        "role": "user",
        "name": "get_borough_iso_code",
        "content": function_response
    })

# 4. Final API Call: Use .parse() to get the final, validated Pydantic object
print("--- 2. Making final .parse() call for structured output ---")
final_response = client.chat.completions.parse(
    model="",
    messages=messages,
    response_format=CodedNarrative,
)

coded_narrative_object = final_response.choices[0].message.parsed

In [ ]:
# --- Display the Final, Structured Result ---
print("--- Final Coded Narrative (Pydantic Object) ---")
pprint(coded_narrative_object.model_dump())

##### Expected Output
When you run the code, you will see the tool execution messages followed by the final structured JSON object.

```json
{
  "summary": "A healthcare worker in the Bronx describes feelings of hopelessness and sadness due to shortages but also experiences incredible solidarity with colleagues.",
  "emotions": [
    "Hopelessness",
    "Sadness",
    "Solidarity"
  ],
  "material_conditions": [
    "running out of everything"
  ],
  "borough_iso_code": "NYC-BX"
}```

---
### Final Takeaways: From Chatbot to Research Partner
Congratulations on completing this workshop! You've gone from making simple API calls to building a sophisticated, multi-step AI agent capable of performing a real research task.

Let's recap the journey:

1. **Prompting**: You started by learning that the "prompt is the program." You learned to guide the LLM's behavior with system prompts and improve its reliability with few-shot examples.
2. **Structured Output**: You then took control of the model's output. You moved from simply asking for JSON to guaranteeing it with Pydantic models and Enums, turning the LLM into a reliable data processor.
3. **Tool Calling**: Finally, you gave the LLM superpowers. By connecting the model's "brain" to the "hands" of Python functions, you enabled it to interact with custom knowledge bases and standardize messy, real-world data.

The most important takeaway is this: an LLM is not just a chatbot. It is a powerful reasoning engine that you can programmatically control and integrate into your research workflow. By combining clear instructions, structured schemas, and custom tools, you can transform it from a simple novelty into a reliable and scalable research partner.

The skills you've learned today are the foundation for building any modern AI-powered application. We encourage you to take the code from this workshop, adapt it to your own research questions, and start building!